# Root Finders in Python
### by [Jason DeBacker](https://jasondebacker.com), October 2024

This Jupyter Notebook illustrates how to do numerical root-finding in Python using the SciPy package.

## The root-finding problem
The general formulation of a root-finder of a system of $N$ non-linear equations can be written as, $\phi(x|z,\theta)=0$, where $f(\cdot)$ is a nonlinear function of variables $x$ and $z$ and parameters $\theta$.  The solution to this system of equations are values $\hat{x}$ that satisfy $\phi(x|z,\theta)=0$:


$$ \hat{x} = x: \quad \phi(x|z,\theta) = 0 $$



## An Example: Minimizing Average Cost

Recall our problem of minimizing average cost.  Given the total cost function $C(x) = a + bx + cx^2$ where output $x \geq 0$ and $a$, $b$, $c$ are positive constants.  Average cost is thus given by $AC(x) = ax^{-1} + b + cx$.  Thus the minimization problem here is:

$$ \min_{x \in \mathbb{R}} : f(x) \quad\text{s.t}\quad x\geq 0 $$

We can write this problem as a scalar root-finding problem where we find the $x$ that satisfies the first order necessary condition of minimizing average cost:

$$
-ax^{-2} + c = 0
$$

The $x$ that solves the FOC is the $x$ that minimizes $AC(x)$.

With this simple problem, we know the $x$, which minimizing $AC(x)$:

$$
\hat{x} = \left(\frac{a}{c}\right)^{1/2}
$$

But let's see how we can find this minimum numerically with a root-finding algorithm.

In [3]:
# imports
import numpy as np
import scipy.optimize as opt

In [1]:
# Write our the FOC of the AC(x) function -- our zero condition
def FOC_AC(x, *params):
    '''
    The first order condition for the average cost function, written
    as a zero condition.

    Args:
    x (scalar): The amount of output
    params (tuple): The cost parameters.  A length-3 tuple.

    Returns:
        error (scalar): The value the FOC (zero in eq'm, but not elsewhere)
    '''
    a, b, c = params
    error = -a * x ** -2 + c

    return error

In [8]:
# set parameters
a, b, c = 1, 2, 3
params = (a, b, c)
# call the function with a root-finder
results = opt.root(FOC_AC, 0.2, args=params)
# compare to analytical solution
print("The analytical solution for x = ", (a/c) ** (1/2))
print("The numerical solution is ", results.x)
print("The difference is ", np.abs((a/c) ** (1/2) - results.x))

The analytical solution for x =  0.5773502691896257
The numerical solution is  [0.57735027]
The difference is  [0.]


## Comparing with a minimization algorithm

Great, we are able to recover the true minimum. 

How does a root-finder compare to numerical optimization used to find the minimum of a function directly?

Let's compare...

In [9]:
# Define the AC function, which we will minimize
def avg_cost(x, *params):
    '''
    This function returns the average cost of output x given
    cost parameters a, b, c.

    Args:
        x (scalar): The amount of output.
        params (tuple): The cost parameters.  A length-3 tuple.

    Returns:
        ac (scalar): The average cost of producing x units of output.
    '''
    a, b, c = params
    ac = a * (x ** -1) + b + c * x

    return ac

In [18]:
# call minimizer of avg_cost and root finder on FOC_AC and compare time to find min
%timeit opt.minimize(avg_cost, 0.2, args=params, method="CG")
%timeit opt.minimize_scalar(avg_cost, bracket = [0.1, 3], args=params)
%timeit opt.root(FOC_AC, 0.2, args=params)

561 μs ± 21.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
71.5 μs ± 1.38 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
49.7 μs ± 237 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Summary

Minimization problems can often be reformulated as root-finding problems (and vice versa).

In this single dimensional problem, the root finder is generally faster.  But this might not generalize to higher dimensional problems.  However, the root-finder is a useful tool to have in your toolbox for solving economic models.

However, note that a potential limitation of the root-finding algorithms in `scipy.optimize.root` are that they do not accept bounds if the function is multi-dimensional -- so bounded root-finding methods need to be defined in a custom fashion (e.g., but recasting it as a minimization problem).